In [ ]:
import requests
from massive import RESTClient
from datetime import datetime, timedelta
import numpy as np
from datetime import date

In [ ]:
class Analyze:
    """
        Makes a dictionary for each day and gives RSI, price value, and number of transactions/shares 
        traded
    """

    def __init__(self, client, day_range):
        self.client = client
        self.today = datetime.today().strftime('%Y-%m-%d')
        self.day_range = day_range

    def get_business_days_back(self):
        """
        Returns the date n business days ago
        """
        current = datetime.now()
        business_days_to_sub = self.day_range
        while business_days_to_sub > 0:
            current -= timedelta(days=1)
            # If it's a weekday (Monday = 0, Sunday = 6)
            if current.weekday() < 5:  # 5 is Saturday, 6 is Sunday
                business_days_to_sub -= 1
        self.from_date = current.strftime('%Y-%m-%d')

    """
        Converts milliseconds timestamp to more readable format
    """
    def convert_timestamp(self, ts):
        return datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d')

    """
        Gets prices for a specific ticker over a set time
    """
    def get_price(self, ticker, multiplier=1, timespan='day'):
        self.time_dict = {}
        for _ in self.client.list_aggs(
                ticker=ticker, 
                multiplier=multiplier, 
                timespan=timespan, 
                from_=self.from_date, 
                to=self.today
        ):
            self.time_dict[self.convert_timestamp(_.timestamp)] = {
                'open': _.open, 
                'close': _.close, 
                'volume': _.volume, 
                'high': _.high, 
                'low': _.low,
                'transactions': _.transactions
            }

    """
        Gets RSI values over time frame and for a specific ticker
    """
    def get_rsi(
            self, 
            ticker, 
            timespan='day',
            adjusted=True, 
            window=14, 
            series_type='close',    
            order='desc', 
            limit=10
    ):
        rsi_data = self.client.get_rsi(
            ticker=ticker,
            timespan=timespan,
            adjusted=adjusted,
            window=window,
            series_type=series_type,
            order=order,
            limit=self.day_range
        )
        for _ in rsi_data.values:
            timestamp = self.convert_timestamp(_.timestamp)
            if timestamp not in self.time_dict:
                print(timestamp)
            else:
                self.time_dict[timestamp]['rsi'] = _.value

In [ ]:
"""
    use api key and set up client
"""
api_key = 'hJQGYwLtVkOmM45yOBeEOg99YqOzJAz9'
client = RESTClient(api_key=api_key)

ticker = 'AAPL'

"""
    instance of class
"""
inst = Analyze(client=client, day_range=21)
inst.get_business_days_back()
inst.get_price(
    ticker=ticker,
    multiplier=1,
)
inst.get_rsi(
    ticker=ticker,
    timespan='day',
    adjusted=True,
    window=14,
    series_type='close',
    order='desc',
)

2025-11-06
2025-10-09


In [ ]:
dict_data = inst.time_dict

In [45]:
dict_data

{'2025-10-09': {'open': 257.805,
  'close': 254.04,
  'volume': 38322012.0,
  'high': 258,
  'low': 253.14,
  'transactions': 536585,
  'rsi': 60.85551086624414},
 '2025-10-10': {'open': 254.94,
  'close': 245.27,
  'volume': 61999098.0,
  'high': 256.38,
  'low': 244,
  'transactions': 908253,
  'rsi': 47.144368780915535},
 '2025-10-13': {'open': 249.38,
  'close': 247.66,
  'volume': 38142942.0,
  'high': 249.69,
  'low': 245.56,
  'transactions': 614923,
  'rsi': 50.422606196149786},
 '2025-10-14': {'open': 246.6,
  'close': 247.77,
  'volume': 35477986.0,
  'high': 248.845,
  'low': 244.7,
  'transactions': 533528,
  'rsi': 50.57454868997531},
 '2025-10-15': {'open': 249.485,
  'close': 249.34,
  'volume': 33893611.0,
  'high': 251.82,
  'low': 247.47,
  'transactions': 528880,
  'rsi': 52.798097715539384},
 '2025-10-16': {'open': 248.25,
  'close': 247.45,
  'volume': 39776974.0,
  'high': 249.04,
  'low': 245.13,
  'transactions': 616258,
  'rsi': 49.88843416308649},
 '2025-10-17